In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns
import re
import librosa
import librosa.display

import IPython.display as ipd

from datetime import datetime, timedelta

import plotly.graph_objects as go
from scipy.interpolate import interp1d 

import librosa
import librosa.display
import IPython.display as ipd
# import noisereduce as nr

# Pytorch
import torch
import torchaudio


sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

In [ ]:
class configuraion():
    '''Configuration File'''
    n_fft = 2048
    
    ################## Spectogram
    #image - Spect
    frame_size = 5 # seg
    frame_step = 5  # seg

    # Spectogram Transformer
    # Default Spec (librosa)
    hop_length  = 128
    frame_size_t  = 256

    #Mel spect (Torch)
    n_mels     = 250
    win_length = 1024
    f_min      = 500
    f_max      = 9000
        
    
    n_show_birds = 5
    
CFG = configuraion()

# BirdCLEF 2022 Competition Warming up !

**under construction**  
The main objective of this notebook is to warm up of for the BirdCLEF 2022 Competion!.Therefore, to understand the basics of the dataset and share!  
To achieve this goal, the following steps are implemented:
- Load data and understand the meta data and traning audio data
- EDA of meta data
- Check some samples of audio and its´ spectograms

# Load Data

- primary_label: Label of main bird\specie
- secondary_label: Labels of bird sounds in background. If none, empty list
- latitude\longitude: geografic location of bird species


In [ ]:
meta = pd.read_csv('../input/birdclef-2022/train_metadata.csv')
meta.head(2)

In [ ]:
eBird = pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
eBird.head(2)

# EDA

In [ ]:
meta['secondary_labels'] = meta['secondary_labels'].apply(lambda x: re.findall(r"'(\w+)'", x))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (18,8),gridspec_kw={'width_ratios': [2, 1]})
plot_temp = meta['secondary_labels'].apply(len).sort_values(ascending = False).reset_index()
sns.countplot('secondary_labels',data = plot_temp ,palette = 'rocket', ax = ax[0])
sns.countplot('secondary_labels',data = plot_temp.query('secondary_labels > 2'),palette = 'rocket', ax = ax[1])

ax[0].set_title(f'Total Number of Birds Sounds on Background', fontdict = {'fontsize':20})
ax[1].set_title(f'3 or more', fontdict = {'fontsize':20})
ax[0].set_xlabel('Number', fontdict = {'fontsize':16})
ax[1].set_xlabel('Number', fontdict = {'fontsize':16})
ax[0].set_ylabel('Count', fontdict = {'fontsize':16})
ax[1].set_ylabel('')

In [ ]:
values = 20
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,8))
sns.barplot(y = 'index', x = 'secondary_labels',data = meta['secondary_labels'].explode().value_counts().head(values).reset_index(),ax = ax,palette = 'rocket')
ax.set_title(f'Top {values} Birds Found on Background as Noise', fontdict = {'fontsize':20})
ax.set_xlabel('Frequency', fontdict = {'fontsize':16})
ax.set_ylabel('Birds Common Name', fontdict = {'fontsize':16})


plt.tight_layout()

In [ ]:
values = 20

top = meta['primary_label'].value_counts().head(values).index
botton = meta['primary_label'].value_counts().tail(values).index

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (18,8))
sns.barplot(y = 'index', x = 'common_name',data = meta['common_name'].value_counts().head(values).reset_index(),ax = ax[0],palette = 'rocket')
sns.barplot(y = 'index', x = 'common_name',data = meta['common_name'].value_counts().tail(values).reset_index(),ax = ax[1], palette = 'viridis')
ax[0].set_title(f'Top {values} Birds', fontdict = {'fontsize':20})
ax[1].set_title(f'Botton {values} Birds', fontdict = {'fontsize':20})
ax[0].set_xlabel('Frequency', fontdict = {'fontsize':16})
ax[1].set_xlabel('Frequency', fontdict = {'fontsize':16})
ax[0].set_ylabel('Birds Common Name', fontdict = {'fontsize':16})
ax[1].set_ylabel('')

plt.tight_layout()

## Map

In [ ]:
df_plot = meta.groupby(['primary_label','latitude', 'longitude']).count().reset_index()[['primary_label','scientific_name','latitude', 'longitude']].rename(columns = {'scientific_name':'count'})
meta = meta.merge(df_plot, on = ['primary_label','latitude', 'longitude'], how = 'left')

values_list = meta['count'].values.tolist()

interpolation = interp1d([1, max(values_list)], [3,20])
radius = interpolation(values_list)
fig = go.Figure(go.Densitymapbox(lat =meta['latitude'],lon = meta['longitude'], radius = radius,z = meta['count']))

fig.update_layout(mapbox_style="open-street-map",height = 800,
                  mapbox = {
                          'center': {'lat': 0, 
                          'lon': 0},
                      'zoom':0
                  })
fig.show()

## Time 

In [ ]:
def round_date(date, delta = 30, th = 10):
    date = date.to_pydatetime()
    x = date.minute
    if ((x >= (delta - th)) & (x < delta)) or (x > (delta + th)):
#         print('Up')
        date = date + (datetime.min - date) % timedelta(minutes = delta)
    elif ((x <= (delta+ th )) & (x > delta)) or (x < (delta - th)):
#         print('down')
        date = date - (date - datetime.min) % timedelta(minutes = delta)

    
    return date.time().strftime("%H:%M")

meta['time_tf']  = pd.to_datetime(meta['time'], errors = 'coerce').dropna().apply(lambda x:round_date(x))
meta.dropna(subset=['time_tf'], inplace = True)

In [ ]:
plt.figure(figsize = (15,8))
sns.countplot(x = 'time_tf', data = meta.sort_values(by = 'time_tf'), palette = 'magma')
plt.xticks(rotation=45)
plt.xlabel('Time', fontdict = {'fontsize':18})
plt.ylabel('Frequency', fontdict = {'fontsize':18})
plt.title('Birds Registers` Time',fontdict = {'fontsize':18})
plt.tight_layout()

## Data Audio Sample

## Top Frequent Birds

In [ ]:
for i in top[:CFG.n_show_birds]:
    print(i)
    path = meta[meta['primary_label'] == i].sample(1,random_state = 666)['filename'].values[0]
    path ='/kaggle/input/birdclef-2022/train_audio/' + path
    ipd.display(ipd.Audio(path))
    

In [ ]:
for i in top[:CFG.n_show_birds]:
    print(i)
    
    path = meta[meta['primary_label'] == i].sample(1,random_state = 666)['filename'].values[0]
    path ='/kaggle/input/birdclef-2022/train_audio/' + path
    
    data, sample_rate = librosa.load(path)
    
    stft = librosa.stft(data, n_fft=CFG.n_fft, hop_length=CFG.hop_length)
    spectrogram = np.abs(stft)
    x = librosa.amplitude_to_db(spectrogram)
    
    
    #mel spectogram
    transfomer = torchaudio.transforms.MelSpectrogram(sample_rate = sample_rate,
                                                     n_fft = CFG.n_fft, 
                                                     win_length = CFG.win_length,
                                                     n_mels = CFG.n_mels,
                                                     f_min = CFG.f_min,
                                                     f_max = CFG.f_max ).double()


    wave = torch.from_numpy(data.copy())
    mel_spectrogram = transfomer(wave)
    
    #PCEN melspectogram
    pcen_spectogram = librosa.pcen(np.array(mel_spectrogram) * (2 ** 31), 
                                  eps = 1e-6,
                                  gain = 0.8,
                                  power = 0.25,
                                  bias = 10, 
                                  sr = sample_rate,
                                  hop_length = CFG.hop_length)
    
    
    fig, ax = plt.subplots(ncols = 2, nrows = 1, figsize = (12,5))
    
    librosa.display.specshow(x, sr=sample_rate, hop_length=CFG.hop_length,ax = ax[0])
    ax[0].set_title("Spectrogram - STFT")
    librosa.display.waveshow(data,ax = ax[1])
    plt.show()
    
    fig, ax1 = plt.subplots(ncols = 2, nrows = 1, figsize = (18,5))
    ax1[0].imshow(librosa.amplitude_to_db(mel_spectrogram))
    ax1[0].set_title("Melspectogram")
    ax1[1].imshow((pcen_spectogram))
    ax1[1].set_title("PCEN-Melspectogram")
    
    
    plt.show()
    
    
    

## Botton Frequent Birds

In [ ]:
for i in botton[:CFG.n_show_birds]:
    print(i)
    path = meta[meta['primary_label'] == i].sample(1,random_state = 666)['filename'].values[0]
    path ='/kaggle/input/birdclef-2022/train_audio/' + path
    ipd.display(ipd.Audio(path))
    

In [ ]:
for i in botton[:CFG.n_show_birds]:
    print(i)
    
    path = meta[meta['primary_label'] == i].sample(1,random_state = 666)['filename'].values[0]
    path ='/kaggle/input/birdclef-2022/train_audio/' + path
    
    data, sample_rate = librosa.load(path)
    
    stft = librosa.stft(data, n_fft=CFG.n_fft, hop_length=CFG.hop_length)
    spectrogram = np.abs(stft)
    x = librosa.amplitude_to_db(spectrogram)
    
    
    #mel spectogram
    transfomer = torchaudio.transforms.MelSpectrogram(sample_rate = sample_rate,
                                                     n_fft = CFG.n_fft, 
                                                     win_length = CFG.win_length,
                                                     n_mels = CFG.n_mels,
                                                     f_min = CFG.f_min,
                                                     f_max = CFG.f_max ).double()


    wave = torch.from_numpy(data.copy())
    mel_spectrogram = transfomer(wave)
    
    #PCEN melspectogram
    pcen_spectogram = librosa.pcen(np.array(mel_spectrogram) * (2 ** 31), 
                                  eps = 1e-6,
                                  gain = 0.8,
                                  power = 0.25,
                                  bias = 10, 
                                  sr = sample_rate,
                                  hop_length = CFG.hop_length)
    
    
    fig, ax = plt.subplots(ncols = 2, nrows = 1, figsize = (12,5))
    
    librosa.display.specshow(x, sr=sample_rate, hop_length=CFG.hop_length,ax = ax[0])
    ax[0].set_title("Spectrogram - STFT")
    librosa.display.waveshow(data,ax = ax[1])
    plt.show()
    
    fig, ax1 = plt.subplots(ncols = 2, nrows = 1, figsize = (18,5))
    ax1[0].imshow(librosa.amplitude_to_db(mel_spectrogram))
    ax1[0].set_title("Melspectogram")
    ax1[1].imshow((pcen_spectogram))
    ax1[1].set_title("PCEN-Melspectogram")
    
    
    plt.show()
    